In [1]:
import torch
import torchvision

from torchvision.transforms import v2
from model_bruno import *
from utils import *
from time import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.ColorJitter(25, 25),
    v2.ToDtype(torch.float32, True),
])

STL10_set = torchvision.datasets.STL10(root='C:/Users/Bruno/OneDrive/Documents/Datasets/STL10', split='train', transform=transforms, download=False)
STL10_loader = torch.utils.data.DataLoader(STL10_set, batch_size=1, shuffle=True,  pin_memory=False)

In [3]:
num_epochs = 3
num_clusters = 10
num_classes = 10
area_penalty = False

model = AttModel(n_classes=num_classes, n_clusters=num_clusters)
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), 1e-4)
criterion = nn.CrossEntropyLoss()

c:\Users\Bruno\miniconda3\envs\pytorch\Lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
model.train()
for epoch in range(num_epochs):
    tic = time()
    avg_loss = 0
    avg_acc = 0
    for x,y in STL10_loader:
        x = x.to(device)
        y = y.to(device)

        pred, _, _ = model(x)
        loss = criterion(pred.view(1,-1),y)

        # if area_penalty:
        #     loss += torch.sum((bboxes[:,-1]*bboxes[:,-2]))/num_clusters**2

        opt.zero_grad()
        loss.backward(retain_graph=False)
        opt.step()
        
        avg_loss += float(loss) / len(STL10_loader)
        avg_acc += (y == pred.argmax(1)).float().mean() / len(STL10_loader)   
    
    toc = time()
    
    print(f'Epoch {epoch+1}/{num_epochs} - {toc-tic:.0f}s - Avg loss: {avg_loss} - Avg acc: {avg_acc}')

Epoch 1/3 - 5729s - Avg loss: 2.3600441992045957 - Avg acc: 0.09940053522586823
Epoch 2/3 - 5667s - Avg loss: 2.361086360502395 - Avg acc: 0.1000005453824997
Epoch 3/3 - 5357s - Avg loss: 2.361150169372711 - Avg acc: 0.1000005453824997


The model appears to not converge. Maybe replace Sigmoid activations for ReLU. It is still only working with batch_size = 1. Also need to evaluate the learning rate and which features should be used for clustering.